### Paxillin movie segmentation and object feature analysis

with Patrick's 2D fixed image pFAK and Zyxin

Liya Ding, 2024.04

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_closing, binary_dilation
from skimage.morphology import disk


from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [2]:
image_2D_2CH_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_pFAK/Control_images'

csv_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Vessp5-LocThrk1_csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Vessp5-LocThrk1_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/pFAK_zyxin_20230423/Vessp5-LocThrk1_seg'

pixel_size = 0.0706
time_point = 0

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [3]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

### for each 3D stack run through MIP and 8th frame

In [4]:
# flag for some optinal parts
flag_plot = True
flag_plot_save = True
flag_seg_save = True
flag_csv_save = True
flag_run_all = True
flag_panel_plot = True

In [5]:
# get a global intensity threshold 
filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    # print(filename)
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    pfak_img = img[1,:,:]
    zyxin_img = img[0,:,:]
    
    intensity_scaling_param = [5,20]
    norm_pfak_img = intensity_normalization(pfak_img, scaling_param=intensity_scaling_param)
    norm_zyxin_img = intensity_normalization(zyxin_img, scaling_param=intensity_scaling_param)
       
    MIP_pfak_img = norm_pfak_img
    MIP_zyxin_img = norm_zyxin_img

    smooth_MIP_zyxin_img = gaussian_filter(MIP_zyxin_img,sigma=4,mode='nearest',truncate=3)
    ### low threshold to get cell masks
    new_cell_mask = smooth_MIP_zyxin_img>0.02
    ### remove isolated small objects (small regions due to noise)
    new_cell_mask = remove_small_objects(new_cell_mask>0, min_size=10000, connectivity=1, in_place=False)        
    # remove holes
    new_cell_mask = ndimage.binary_fill_holes(new_cell_mask)

    if filenameID ==0:
        all_pfak_img = np.zeros([len(filenames),pfak_img.shape[0],pfak_img.shape[1]])
        all_zyxin_img = np.zeros([len(filenames),pfak_img.shape[0],pfak_img.shape[1]])
        all_cell_mask = np.zeros([len(filenames),pfak_img.shape[0],pfak_img.shape[1]])
    all_pfak_img[filenameID,:,:] = pfak_img
    all_zyxin_img[filenameID,:,:] = zyxin_img
    all_cell_mask[filenameID,:,:] = new_cell_mask

    
global_threshold  = all_pfak_img[all_cell_mask>0].mean() + 1*all_pfak_img[all_cell_mask>0].std()

In [6]:
zyxin_data = all_zyxin_img.flatten()

In [7]:
pfak_data = all_pfak_img.flatten()

In [8]:
np.percentile(zyxin_data,0.1)

92.0

In [ ]:
np.percentile(zyxin_data,99.9)

In [ ]:
np.percentile(pfak_data,0.1)

In [ ]:
np.percentile(pfak_data,99.9)

In [ ]:
from scipy.ndimage import gaussian_filter

# get a global intensity threshold 
filenames = [x for x in os.listdir(image_2D_2CH_dir) if os.path.isfile(os.path.join(image_2D_2CH_dir, x)) and ('.tif' in x)]
norm_image_2D_2CH_dir =  '/mnt/d/lding/zyxin/data/240423_fixed_EGFP_zyxin_pFAK/norm_pny_Control_images'

for filenameID in range(len(filenames)):
    filename = filenames[filenameID]
    # print(filename)
    img = tifffile.imread(os.path.join(image_2D_2CH_dir,filename))
    pfak_img = img[1,:,:]
    zyxin_img = img[0,:,:]
  
    smooth_MIP_zyxin_img = gaussian_filter(zyxin_img,sigma=2,mode='nearest',truncate=3)    
    new_cell_mask = smooth_MIP_zyxin_img>104
    ### remove isolated small objects (small regions due to noise)
    new_cell_mask = remove_small_objects(new_cell_mask>0, min_size=10000, connectivity=1, in_place=False)     
    new_cell_mask = binary_closing(new_cell_mask, disk(11))   
    # remove holes
    new_cell_mask = ndimage.binary_fill_holes(new_cell_mask)
      
    pfak_img = (pfak_img - 99.0) / (303-99)
    zyxin_img = (zyxin_img - 92.0) / (608-92)


    new_img = np.zeros([8,img.shape[1],img.shape[2]])

    new_img[6,:,:] = zyxin_img
    new_img[7,:,:] = pfak_img
    new_img[4,:,:] = new_cell_mask

    np.save(os.path.join(norm_image_2D_2CH_dir, 'norm_'+filename[:-4]+'.npy'),new_img)   
        
    


In [12]:
AA = ((pfak_data - 92.0) / (608-92))

In [13]:
AA.mean()

0.050914162449798496

In [ ]:
channel_check =np.load('/mnt/d/lding/zyxin/data/data_pny/TractionData_21_02_17_ZyxinActin/cell_3/Disp_Trac_CellMask_ForceMask_Actin_0.npy')

In [ ]:
plt.imshow(channel_check[7],interpolation=None)